# Crop Fire Classification Function
This file contains a function which can classify fire points into agricultural and non-agricultural fires: <br>
Given in any area, this function will classify the fire points.<br>
<br>
All libraries has been imported which are required to create function.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import geopandas as gpd                 # To handle geospatial data
import pandas as pd                     # To handle any other data
import rasterio                         # To handle geosaptial raster files

### Reading csv file for FIRMS fire points.
FIRMS data for both MODIS and VIIRS S-NPP can be downloaded in the format of Country Yearly Summary [.csv format](https://firms.modaps.eosdis.nasa.gov/country/). MODIS is available from 2000 to 2020, whereas VIIRS S-NPP is available from 2012 to the previous year(i.e., 2021).
<br>
<br>
Boundary can be downloaded with respect to FIRMS data. Both boundary shapefile and FIRMS data can be read as geopandas, later FIRMS data can be clipped with given boundary:

In [ ]:
viirs = pd.read_csv('ENTER THE CSV FILE NAME AND LOCATION HERE')  #ENTER THE CSV FILE NAME AND LOCATION HERE
adm_name = gpd.read_file('ENTER THE GEOJSON/SHP FILE NAME AND LOCATION HERE')

fire_pts = gpd.GeoDataFrame(viirs,                        #Converting fire points into dataframe
                        geometry=gpd.points_from_xy(
                            viirs.longitude,
                            viirs.latitude),
                        crs=4326)

gdf = fire_pts.clip(adm_name)                   #Clipping fire points with boundary
gdf = fire_pts.sort_values(by=["acq_date"])     #Sorting points by date

### Download and Read LULC file
ESRI's 10m LULC 2017-2021 maps can [downloaded](https://www.arcgis.com/apps/instant/media/index.html?appid=fc92d38533d440078f17678ebc20e8e2) tile wise according to given area of interest. This file can be read as an pandas using rasterio, and FIRMS clipped points can be classified accordinf to each pixel value of LULC raster file:

In [ ]:
#list of coordinates for all points
coords = [(x,y) for x, y in zip(gdf.longitude, gdf.latitude)]

# Open the raster and store metadata
src = rasterio.open('ENTER LULC FILE NAME AND LOCATION HERE')

# Sample the raster at every point location and store values in GeoDataFrame
gdf['Class'] = [x[0] for x in src.sample(coords)]                #define each fire points the classes of respective LULC pixels

#List of all classes in LULC raster
classes = list(gdf.Class.unique())
# Crop class - 4, flooded vegetation (like paddy) - 5
crop_class = [4,5]
non_crop_class = classes - crop_class

gdf['Class'] = gdf['Class'].replace(non_crop_class,0)       #non-agricultural fires as 0
gdf['Class'] = gdf['Class'].replace(crop_class,1)           #agricultural fires as 1


gdf = gdf.reset_index()
del gdf['index']
gdf = gdf.reset_index()
gdf.rename(columns = {'index':'fireID'}, inplace = True)   #Give each fire points a unique ID

### Merge all in one function:

Bring down all command together to create a function which takes FIRMS data, boundary shapefile and LULC as an input and create a dataframe as an output which contains all fire as a agricultural and non-agricultural classes:

In [2]:
def fire_class (firms, boundary, lulc):
    viirs = pd.read_csv(firms)  #ENTER THE CSV FILE NAME AND LOCATION HERE
    adm_name = gpd.read_file(boundary)

    fire_pts = gpd.GeoDataFrame(viirs,                        #Converting fire points into dataframe
                            geometry=gpd.points_from_xy(
                                viirs.longitude,
                                viirs.latitude),
                            crs=4326)

    gdf = fire_pts.clip(adm_name)                   #Clipping fire points with boundary
    gdf = fire_pts.sort_values(by=["acq_date"])     #Sorting points by date


    #list of coordinates for all points
    coords = [(x,y) for x, y in zip(gdf.longitude, gdf.latitude)]

    # Open the raster and store metadata
    src = rasterio.open(lulc)

    # Sample the raster at every point location and store values in GeoDataFrame
    gdf['Class'] = [x[0] for x in src.sample(coords)]                #define each fire points the classes of respective LULC pixels

    #List of all classes in LULC raster
    classes = list(gdf.Class.unique())
    # Crop class - 4, flooded vegetation (like paddy) - 5
    crop_class = [4,5]
    non_crop_class = classes - crop_class

    gdf['Class'] = gdf['Class'].replace(non_crop_class,0)       #non-agricultural fires as 0
    gdf['Class'] = gdf['Class'].replace(crop_class,1)           #agricultural fires as 1


    gdf = gdf.reset_index()
    del gdf['index']
    gdf = gdf.reset_index()
    gdf.rename(columns = {'index':'fireID'}, inplace = True)   #Give each fire points a unique ID

#### Function can be called as:

In [ ]:
fire_classified_dataframe = fire_class('firms/data/address','boundary/shape/address','lulc/raster/address')